In [2]:
# 03_iterative_refinement.py
import os
import json
import re
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

load_dotenv()

# Configuration
QUALITY_THRESHOLD = 4.0  # Minimum average score (1-5) to pass
MAX_ITERATIONS = 3       # Prevent infinite loops

search_tool = SerperDevTool()

# ============================================================
# AGENTS (same as before)
# ============================================================

researcher = Agent(
    role="Senior Research Analyst",
    goal="Find the most relevant academic papers on a topic",
    backstory="""You are a meticulous researcher who finds high-quality 
    academic sources from top venues.""",
    tools=[search_tool],
    verbose=True
)

summarizer = Agent(
    role="Technical Writer", 
    goal="Create clear, accurate summaries of research papers",
    backstory="""You are a skilled technical writer. When given feedback, 
    you incorporate it thoughtfully to improve your work. You don't get 
    defensive about criticism—you use it.""",
    verbose=True
)

critic = Agent(
    role="Quality Reviewer",
    goal="Evaluate and score summaries, providing actionable feedback",
    backstory="""You are a demanding but fair editor. You provide numerical 
    scores AND specific feedback. You notice when improvements are made 
    across iterations and acknowledge progress.""",
    verbose=True
)

# ============================================================
# TASKS
# ============================================================

research_task = Task(
    description="""Find the top 3 papers on: {topic}
    
    For each paper provide: title, authors, year/venue, and key findings.
    """,
    expected_output="Structured list of 3 papers with metadata and findings.",
    agent=researcher
)

summarize_task = Task(
    description="""Summarize the research papers provided.
    
    {previous_feedback}
    
    For each paper, write a 3-4 sentence summary covering:
    - Main contribution
    - Methodology  
    - Key results
    - Broader implications
    """,
    expected_output="Three clear, accurate paper summaries.",
    agent=summarizer
)

critique_task = Task(
    description="""Evaluate each summary on a 1-5 scale for:
    - Accuracy: Does it correctly represent the paper?
    - Clarity: Is it understandable?
    - Completeness: Are key points covered?
    
    Return your evaluation as JSON:
```json
    [
        {{"paper": "Paper Title", "accuracy": X, "clarity": X, "completeness": X, 
          "feedback": "Specific suggestions for improvement"}}
    ]
```
    
    Be precise in your scores. 5 means excellent, not just "acceptable."
    """,
    expected_output="JSON array with scores and feedback for each summary.",
    agent=critic
)

# ============================================================
# ORCHESTRATION LOGIC
# ============================================================

def extract_scores(critique_output):
    """Parse the critic's JSON output to extract scores."""
    try:
        # Find JSON in the output
        json_match = re.search(r'\[.*\]', critique_output, re.DOTALL)
        if json_match:
            evaluations = json.loads(json_match.group())
            scores = []
            for eval in evaluations:
                avg = (eval.get('accuracy', 0) + eval.get('clarity', 0) + 
                       eval.get('completeness', 0)) / 3
                scores.append(avg)
            return sum(scores) / len(scores) if scores else 0
    except (json.JSONDecodeError, KeyError, TypeError) as e:
        print(f"⚠️  Could not parse critique scores: {e}")
    return 0

def extract_feedback(critique_output):
    """Extract written feedback from critique for next iteration."""
    try:
        json_match = re.search(r'\[.*\]', critique_output, re.DOTALL)
        if json_match:
            evaluations = json.loads(json_match.group())
            feedback_items = [eval.get('feedback', '') for eval in evaluations]
            return "Previous feedback to address:\n- " + "\n- ".join(feedback_items)
    except:
        pass
    return ""

def run_research_pipeline(topic):
    """Run the full pipeline with iterative refinement."""
    
    # Phase 1: Research (runs once)
    print(f"\n🔍 Phase 1: Researching '{topic}'...")
    research_crew = Crew(
        agents=[researcher],
        tasks=[research_task],
        process=Process.sequential,
        verbose=True
    )
    research_result = research_crew.kickoff(inputs={"topic": topic})
    
    # Phase 2: Summarize + Critique loop
    previous_feedback = ""
    
    for iteration in range(1, MAX_ITERATIONS + 1):
        print(f"\n📝 Phase 2, Iteration {iteration}: Summarizing and critiquing...")
        
        # Update task with any previous feedback
        summarize_task.description = f"""Summarize the research papers provided.
        
        {previous_feedback}
        
        For each paper, write a 3-4 sentence summary covering:
        - Main contribution
        - Methodology  
        - Key results
        - Broader implications
        
        Research to summarize:
        {research_result.raw}
        """
        
        # Run summarizer + critic
        refine_crew = Crew(
            agents=[summarizer, critic],
            tasks=[summarize_task, critique_task],
            process=Process.sequential,
            verbose=True
        )
        refine_result = refine_crew.kickoff()
        
        # Check quality
        avg_score = extract_scores(refine_result.raw)
        print(f"\n📊 Iteration {iteration} average score: {avg_score:.2f}")
        
        if avg_score >= QUALITY_THRESHOLD:
            print(f"✅ Quality threshold ({QUALITY_THRESHOLD}) met!")
            return refine_result.raw
        
        if iteration < MAX_ITERATIONS:
            previous_feedback = extract_feedback(refine_result.raw)
            print(f"⚠️  Score {avg_score:.2f} below threshold. Refining...")
        else:
            print(f"🚨 Max iterations reached. Proceeding with current output.")
            return refine_result.raw
    
    return refine_result.raw

if __name__ == "__main__":
    topic = "large language model alignment"
    final_output = run_research_pipeline(topic)
    
    print("\n" + "="*60)
    print("FINAL OUTPUT")
    print("="*60)
    print(final_output)


🔍 Phase 1: Researching 'large language model alignment'...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 68ecf0bf-0608-4400-a627-c1580e522267                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Find the top 3 papers on: large language model alignment                                                 │
│                                                                                                                 │
│      For each paper provide: title, authors, year/venue, and key findings.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: I need to search for relevant academic papers on the topic of large language model alignment.         │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quality Reviewer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```json                                                                                                        │
│  [                                                                                                              │
│      {                                                                                                          │
│          "paper": "Training large language models on narrow tasks can lead to broad generalization",            │
│          "accuracy": 5,                                                                                         │
│          "clarity": 5,                                                                                          │
│          "completeness": 5,                                                                                     │
│          "feedback": "The summary accurately captures the core findings and implications of the paper. It is    │
│  well-structured and presents the information clearly and comprehensively."                                     │
│      },                                                                                                         │
│      {                                                                                                          │
│          "paper": "Aligning Large Language Models with Human: A Survey",                                        │
│          "accuracy": 4,                                                                                         │
│          "clarity": 4,                                                                                          │
│          "completeness": 4,                                                                                     │
│          "feedback": "While the summary covers the main focus and findings, it could enhance clarity by         │
│  briefly outlining some of the specific alignment methodologies mentioned. This would provide a more complete   │
│  picture of the survey's findings."                                                                             │
│      },                                                                                                         │
│      {                                                                                                          │
│          "paper": "Aligning large language models with human preferences using [Link issue]",                   │
│          "accuracy": 4,                                                                                         │
│          "clarity": 3,                                                                                          │
│          "completeness": 4,                                                                                     │
│          "feedback": "The summary presents key points but lacks clarity in articulating the specifics of the    │
│  identified obstacles and the importance of these findings. Elaboration on how the outlined areas for           │
│  improvement relate to broader implications would enhance understanding."                                       │
│      }                                                                                                          │
│  ]                                                                                                              │
│  ```                                                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f4b3f8ba-4a2c-4035-9952-76e936ab5e45                                                                     │
│  Agent: Quality Reviewer                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e122ce86-8717-428f-a105-386a2e2de27f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```json                                                                                          │
│  [                                                                                                              │
│      {                                                                                                          │
│          "paper": "Training large language models on narrow tasks can lead to broad generalization",            │
│          "accuracy": 5,                                                                                         │
│          "clarity": 5,                                                                                          │
│          "completeness": 5,                                                                                     │
│          "feedback": "The summary accurately captures the core findings and implications of the paper. It is    │
│  well-structured and presents the information clearly and comprehensively."                                     │
│      },                                                                                                         │
│      {                                                                                                          │
│          "paper": "Aligning Large Language Models with Human: A Survey",                                        │
│          "accuracy": 4,                                                                                         │
│          "clarity": 4,                                                                                          │
│          "completeness": 4,                                                                                     │
│          "feedback": "While the summary covers the main focus and findings, it could enhance clarity by         │
│  briefly outlining some of the specific alignment methodologies mentioned. This would provide a more complete   │
│  picture of the survey's findings."                                                                             │
│      },                                                                                                         │
│      {                                                                                                          │
│          "paper": "Aligning large language models with human preferences using [Link issue]",                   │
│          "accuracy": 4,                                                                                         │
│          "clarity": 3,                                                                                          │
│          "completeness": 4,                                                                                     │
│          "feedback": "The summary presents key points but lacks clarity in articulating the specifics of the    │
│  identified obstacles and the importance of these findings. Elaboration on how the outlined areas for           │
│  improvement relate to broader implications would enhance understanding."                                       │
│      }                                                                                                          │
│  ]                                                    


📊 Iteration 1 average score: 4.22
✅ Quality threshold (4.0) met!

FINAL OUTPUT
```json
[
    {
        "paper": "Training large language models on narrow tasks can lead to broad generalization",
        "accuracy": 5,
        "clarity": 5,
        "completeness": 5,
        "feedback": "The summary accurately captures the core findings and implications of the paper. It is well-structured and presents the information clearly and comprehensively."
    },
    {
        "paper": "Aligning Large Language Models with Human: A Survey",
        "accuracy": 4,
        "clarity": 4,
        "completeness": 4,
        "feedback": "While the summary covers the main focus and findings, it could enhance clarity by briefly outlining some of the specific alignment methodologies mentioned. This would provide a more complete picture of the survey's findings."
    },
    {
        "paper": "Aligning large language models with human preferences using [Link issue]",
        "accuracy": 4,
        "clarity

In [ ]:
# ==========================================
# 1. SETUP & STANDARDS
# ==========================================
# ---------------------------------------------------------------
# Build Your First Agent — Multi-Agent Research Assistant
# Agents: Researcher → Summarizer ↔ Critic ↔ Judge
# ---------------------------------------------------------------

from crewai import Agent, LLM
import re
import os
from dotenv import load_dotenv
import os

# This loads your API keys from a hidden .env file so they aren't exposed in the code.
load_dotenv()

from crewai import Agent, Task, Crew, LLM
import re # 're' stands for Regex, used to find patterns (like numbers) in text.

# We set a "temperature" of 0.4. 
# Lower temperature = more focused/predictable. Higher = more creative.
llm = LLM(model="gpt-4o-mini", temperature=0.4)

# THE QUALITY GATES
# These are the "passing grades" for our digital employees.
MIN_ACCEPTABLE_SCORE = 4       # If the Critic gives a score below 4/5, we redo it.
MIN_JUDGE_CONFIDENCE = 0.8     # If the Judge is less than 80% confident, we redo it.
MAX_RETRIES = 2                # We only allow them to try twice so they don't loop forever.

# ==========================================
# 2. THE EMPLOYEE ROSTER
# ==========================================
# 1. The Researcher: Digs up the raw info.
researcher = Agent(
    role="ResearcherAgent",
    goal="Find top 3 recent papers for a given topic.",
    backstory="Expert academic researcher skilled in literature discovery.",
    llm=llm,
)

# 2. The Writer: Condenses the info.
summarizer = Agent(
    role="SummarizerAgent",
    goal="Summarize papers into concise factual abstracts (<150 words).",
    backstory="Academic summarizer emphasizing clarity and novelty.",
    llm=llm,
)

# 3. The Editor: Grades the Writer's work.
critic = Agent(
    role="CriticAgent",
    goal="Rate summaries (1–5) for novelty and clarity; provide feedback.",
    backstory="Peer reviewer offering constructive feedback.",
    llm=llm,
)

# 4. The Compliance Officer: Checks for lies (hallucinations) or plagiarism.
judge = Agent(
    role="JudgeAgent",
    goal="Assess factual accuracy & plagiarism risk (confidence 0–1).",
    backstory="Integrity checker ensuring factual and original writing.",
    llm=llm,
)

# ==========================================
# 3. HELPER TOOLS
# ==========================================
# Robots reply in text (e.g., "I give this a score of 4 out of 5").
# We need math to make decisions. These functions extract the numbers from the text.

def parse_avg_score(text: str) -> float:
    # Finds all numbers 1-5 in the text and calculates the average.
    nums = [int(x) for x in re.findall(r"\b[1-5]\b", text)]
    return sum(nums) / len(nums) if nums else 0

def parse_confidence(text: str) -> float:
    # Finds decimal numbers (like 0.95) to measure confidence.
    vals = [float(v) for v in re.findall(r"0\.\d+|1\.0", text)]
    return sum(vals) / len(vals) if vals else 0.0


# ==========================================
# 4. THE "SUBCREW" (The Writer's Room)
# ==========================================
# This function isolates the Writer and Critic. 
# We group them so we can run them repeatedly without re-running the Researcher.
def run_subcrew_once(input_text: str):
    """Run Summarizer + Critic sequentially as a subcrew and return outputs + score."""

    # Task A: Write the summary.
    task_summarize = Task(
        description=f"Summarize these papers (<150 words each):\n{input_text}",
        agent=summarizer,
        expected_output="Concise and factual summaries under 150 words each."
    )

    # Task B: Grade the summary.
    task_critic = Task(
        description=(
            "Rate each summary on novelty and clarity (1–5) and give improvement feedback.\n"
            "Return JSON: [{'paper':'...','novelty':x,'clarity':y,'feedback':'...'}]\n\n"
            "Use the summaries below:\n{output_of_task_summarize}"
        ),
        agent=critic,
        expected_output="JSON list with numeric ratings (1–5) and feedback comments."
    )

    # Create a temporary team just for this cycle.
    subcrew = Crew(
        agents=[summarizer, critic],
        tasks=[task_summarize, task_critic],
        process="sequential",
    )

    subcrew.kickoff() 
    
    # We capture the text output and the calculated score.
    summarize_output = task_summarize.output
    critic_output = task_critic.output
    avg_score = parse_avg_score(str(critic_output))
    
    return summarize_output, critic_output, avg_score


# ==========================================
# 5. MAIN MANAGER LOGIC
# ==========================================
def run_research_assistant(topic="Large Language Model Alignment"):
    
    # --- PHASE 1: RESEARCH ---
    # This only happens once. We don't need to re-research if the writing is bad.
    print(f"\n🔍 [1] ResearcherAgent: fetching papers on '{topic}'...\n")

    research_prompt = (
        f"Find 3 recent academic papers about '{topic}' from arXiv or Semantic Scholar. "
        f"For each, list title, authors, and a short abstract."
    )
    # Direct call to the LLM to get raw data quickly.
    raw_papers = researcher.llm.call(research_prompt)
    print("✅ Research complete.\n")

    # Prepare data for the loop
    summarization_input = raw_papers
    summarize_output = critic_output = judge_output = None

    # --- PHASE 2: THE LOOP (Write -> Critique -> Judge -> Repeat) ---
    for attempt in range(1, MAX_RETRIES + 1):
        print(f"\n🧠 Iteration {attempt}: Running Summarizer + Critic SubCrew...\n")

        # 1. Run the Writer's Room (Writer + Editor)
        summarize_output, critic_output, avg_score = run_subcrew_once(summarization_input)
        print(f"📊 Critic Average Score: {avg_score}\n")

        # 2. CHECKPOINT A: Is the writing good enough?
        if avg_score < MIN_ACCEPTABLE_SCORE:
            print(f"⚠️ Score {avg_score:.2f} < {MIN_ACCEPTABLE_SCORE}. Re-running SubCrew right now...\n")
            
            # THE FEEDBACK LOOP:
            # We create new instructions that include the *previous* bad work 
            # and the *critic's complaints*, asking the agent to fix it.
            refined_input = (
                f"{summarize_output}\n\nCritic feedback:\n{critic_output}\n\n"
                "Revise summaries to improve clarity and novelty."
            )
            
            # Recursive call: Run the team again immediately with the new instructions.
            summarize_output, critic_output, avg_score = run_subcrew_once(refined_input)
            print(f"🌀 Re-run complete. New average score: {avg_score}\n")

        # 3. CHECKPOINT B: The "Judge" (Compliance)
        # Only reached if we passed the Critic or ran out of retries.
        print("✅ Critic threshold met (or forced). Proceeding to JudgeAgent...\n")

        judge_prompt = (
            f"You are {judge.role}. {judge.goal}\n"
            "Evaluate factual accuracy and potential plagiarism in these summaries.\n"
            "Return JSON: [{'paper':'...','confidence':0.xx,'flag':bool,'comment':'...'}]\n\n"
            f"{summarize_output}"
        )
        judge_output = judge.llm.call(judge_prompt)
        judge_conf = parse_confidence(judge_output)
        print(f"🧮 Judge Confidence: {judge_conf}\n")

        # 4. FINAL DECISION
        if judge_conf >= MIN_JUDGE_CONFIDENCE:
            print("🎯 Judge confidence acceptable → workflow complete.\n")
            break # Success! Exit the loop.
        else:
            # Failure! The Judge rejected it.
            print(f"🚨 Judge confidence {judge_conf:.2f} < {MIN_JUDGE_CONFIDENCE}.  \n")
            # We update the input with Judge's complaints and loop back to the start (Iteration 2).
            summarization_input = (
                f"{summarize_output}\n\nJudge feedback:\n{judge_output}\n\n"
                "Revise summaries to improve factual accuracy and originality."
            )
        
    # --- PHASE 3: FINAL REPORT ---
    print("🏁 [Final Summaries]\n", summarize_output)
    print("\n🧾 [Critic Feedback]\n", critic_output)
    print("\n⚖️ [Judge Output]\n", judge_output)

# -------------------- 6️⃣ Run --------------------
if __name__ == "__main__":
    run_research_assistant()